In [1]:
import torch # 기본 파이토치 기능
import torch.nn as nn # nn 모듈 기능
import torch.nn.functional as F # 기본 신경망 함수
import torch.optim as optim # 최적화
from torchvision import datasets, transforms # 데이터셋 처리
import matplotlib.pyplot as plt # 데이터 시각화

# ========================
# 1. 데이터 로드 및 전처리
# ========================
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # 평균 0.5, 표준편차 0.5로 정규화
    ])
    # 학습 데이터 셋 다운로드 60000개 / 저장 위치는 root의 data 폴더 
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    # 테스트 데이터 셋 다운로드 10000개 / 저장 위치는 root의 data 폴더 
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    
    # 6만개 샘플들을 64 배치 사이즈 설정 및 셔플 설정하여 랜덤하게 train_loader에 담는다. 
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    # 1만개 샘플들을 64 배치 사이즈 설정 및 셔플 미설정하여 순서대로 test_loader에 담는다. 
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

Matplotlib is building the font cache; this may take a moment.


### Data loading & preprocessing

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # C1: 합성곱 층 (입력: 1x32x32 → 출력: 6x28x28)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        # S2: 평균 풀링 층 (출력: 6x14x14)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        # C3: 합성곱 층 (출력: 16x10x10)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        # S4: 평균 풀링 층 (출력: 16x5x5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        # C5: 합성곱 층 (출력: 120x1x1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0)
        # F6: 완전 연결층 (출력: 84)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        # Output: 완전 연결층 (출력: 10)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # 입력 크기: (batch_size, 1, 28, 28)
        # LeNet-5는 32x32 입력을 기대하지만, MNIST는 28x28이므로 패딩을 추가
        x = F.pad(x, (2, 2, 2, 2))  # 패딩 추가하여 32x32로 변환
        x = F.relu(self.conv1(x))    # C1
        x = self.pool1(x)            # S2
        x = F.relu(self.conv2(x))    # C3
        x = self.pool2(x)            # S4
        x = F.relu(self.conv3(x))    # C5 120x1x1 
        x = x.view(x.size(0), -1)    # Flatten 64x120의 2차원 텐서로 변경
        x = F.relu(self.fc1(x))      # F6
        x = self.fc2(x)              # Output
        return x

### LeNet5 Modeling
#### 1 - 1st conv : 커널 사이즈 5x5
- 입력 : 1채널의 32x32 이미지
- 출력 : 6채널의 28x28 이미지
- **stride=1**은 컨볼루션 연산 시 건너 뛰는 간격으로, 보통 모든 픽셀에 대해 컨볼루션 수행하므로 1로 설정함

#### RELU 함수 적용1

#### 2 - 1st pool : 커널 사이즈 2x2 
- 입력 : 6채널의 28x28 이미지
- 출력 : 6채널의 14x14 이미지

#### 3 - 2nd conv : 커널 사이즈 5x5 
- 입력 : 6채널의 14x14 이미지 
- 출력 : 16채널의 10x10 이미지 (padding 없이 5x5 커널을 사용했으므로 상하좌우로 2라인씩 줄어들어 10x10이 됨)
-> 6x14x14에 **16개의 5x5 커널**과 컨볼루션 연산 하면 16x10x10이 됨

#### RELU 함수 적용2

#### 4 - 2nd pool : 커널 사이즈 2x2
- 입력 : 16채널의 10x10 이미지
- 출력 : 16채널의 5x5 이미지

#### 5 - 3rd conv : 커널 사이즈 5x5
- 입력 : 16채널의 5x5 이미지 
- 출력 : 120채널의 1x1 이미지

#### RELU 함수 적용3

#### 6 - FCL1
- 입력 : 120 차원 벡터 [[x1, x2, x3, ... ,x120]]
- 출력 : 84 차원 벡터 [[x1, x2, x3, ... ,x84]]

#### RELU 함수 적용4 

#### 7 - FCL2
- 입력 : 84 차원 벡터 [[x1, x2, x3, ... ,x84]]
- 출력 : 10 차원 벡터 [[x1, x2, x3, ... ,x10]] **-> logit**

---
#### 멀티 채널에서의 컨볼루션 연산 
입력: 32×32x3
필터: 3x5×5, 총 6개.
출력: 28x28x6
하나의 필터는 입력 데이터의 모든 채널(3채널)에 대해 합성곱 연산을 수행한 후, 결과를 합산하여 하나의 피처맵 생성.
6개의 필터가 독립적으로 작동하여 총 6개의 피처맵 생성. 

---
#### Flatten
- Fully Connected Layer를 통과하기 전에 Flatten 처리를 함
- x.view(x.size(0), -1)
- x는 120x1x1의 3차원 텐서이고, x.size(0)는 배치 사이즈를 의미 64
- 64x120 크기의 2차원 텐서로 평탄화 시킴

In [ ]:
# ========================
# 3. 학습 함수
# ========================
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device) 
    # 모델의 (정적 초기화 레이어의 파라미터 및 버퍼) cpu 디바이스로 보내짐 (근데 원래 cpu에서 생성되었음)
    # 모델이 원래 cpu에서 생성되었으므로, 사실 위치를 변경하지 않으나, 코드의 일관성, 호환성, 안정성, 명확성을 위하여 코드로 명시하는 것이 좋음
    # 추후 GPU에서 학습 시 모델을 GPU로 보내기 위해 값을 변경해주어야 함 

    for epoch in range(num_epochs):
        model.train() # 모델을 training mode로 전환
        total_loss = 0 # loss 값 초기화 
        for images, labels in train_loader: # 훈련 셋의 배치 단위로 images, lables 반환이 반복됨 
            images, labels = images.to(device), labels.to(device) # 데이터x도 cpu 디바이스로 보냄. (마찬가지로 GPU로 변경 시 GPU로 보내짐)

            # Forward pass
            outputs = model(images) # images만 모델에 입력
            loss = criterion(outputs, labels) # 모델 출력 결과와 정답 비교하여 loss 계산 

            # Backward pass and optimization
            optimizer.zero_grad() # 옵티마이저의 기울기 초기화 
            loss.backward() # 손실에 대한 기울기 계산 
            optimizer.step() # 옵티마이저의 최적화 단계 수행

            total_loss += loss.item() # 손실 누적 값 계산 
        
        # 한 Epoch에서의 평균 손실 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

In [ ]:
# ========================
# 4. 평가 함수
# ========================
def evaluate_model(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images) # output = 64 x 10 (배치 사이즈 x 10차원 행벡터)
            _, predicted = torch.max(outputs, 1) # torch.max의 반환 값은 튜플 : 각 행의 최대값, 각 행의 최대값 인덱스
            total += labels.size(0) # 현재 샘플 갯수 (배치 단위로 더함 64+64+ ... )
            correct += (predicted == labels).sum().item() # (predicted == labels)는 Boolean 값이고, 이것 또한 batch size 단위로 계산됨. 즉 64 차원의 Boolean값이 저장된 텐서임

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

In [ ]:
# ========================
# 5. 예측 시각화
# ========================
def visualize_predictions(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    data_iter = iter(test_loader) # test_loader를 iterator로 변환 (데이터를 직접 순회하기 위해)
    images, labels = next(data_iter) # 이터레이터를 생성하고 바로 next()를 호출하면 맨 처음 값부터 가져 옴
    images, labels = images[:5].to(device), labels[:5].to(device) # 처음부터 5까지 이미지랑 레이블을 디바이스로 보냄

    # 모델 예측
    outputs = model(images) # 모델에 이미지를 입력하고 아웃풋을 저장
    _, preds = torch.max(outputs, 1) # 예측값을 preds 변수에 저장 

    # 시각화
    fig, axes = plt.subplots(1, 5, figsize=(12, 3)) # 
    for idx, ax in enumerate(axes):
        ax.imshow(images[idx].cpu().squeeze(), cmap='gray')
        ax.set_title(f'Label: {labels[idx].item()}\nPred: {preds[idx].item()}')
        ax.axis('off')
    plt.show()

In [ ]:
# ========================
# 6. 메인 실행
# ========================
if __name__ == "__main__":
    # 하이퍼파라미터
    batch_size = 64
    num_epochs = 5
    learning_rate = 0.001

    # 데이터 로드
    train_loader, test_loader = load_data(batch_size)

    # 모델 초기화
    #model = CNN()
    model = LeNet5()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 모델 학습
    train_model(model, train_loader, criterion, optimizer, num_epochs)

    # 모델 평가
    evaluate_model(model, test_loader)

    # 예측 시각화
    visualize_predictions(model, test_loader)